Create a file with diurnal cycle data from closest grid point
===

Import required modules and set file names
----

In [1]:
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

In [2]:
# modelDataFN = "daily/ei.oper.an.sfc.regn128sc.tas.19960101-20141231.nc"
# suff = "19960101-20171231"
yrs = "1996-2005"
modelDataFN = "tas_"+yrs+"_diurnal.nc"
modelMetaFN = "erai_geog_sub.nc"
newNCFN = "erai_tas_"+yrs+"_diurnal_closest.nc"

Load/define ERA Interim metadata
----

In [3]:
M = Model(modelMetaFN)
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

1d dataset


In [4]:
# g = M.getMeta()
# g[-1]

Load AWS metadata
----

In [5]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points
-----

In [6]:
C = []
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
# for awsIX in ((0,)):
    print str(awsIX)
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    C.append( closest )
#     if awsIX == 5:
#         print closest.dtype
#         print closest

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


Process tas data
----

In [7]:
print modelDataFN
D = Model(None, modelDataFN)

# time, timeCF = D.loadData( "time" )
# print time[0], time[-1]
# dfTime = pd.Series( time, name="Time")

month = np.array( D.loadData( "month" ) )
hour = np.array( D.loadData( "hour" ) )
tas = np.array( D.loadData( "tas" ) )

tas_1996-2005_diurnal.nc


In [8]:
nMon, nHour, nLat, nLon = tas.shape
print nMon, nHour, nLat, nLon

Tnew = np.empty( (nMon, nHour, nSites), dtype = 'float' )

for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
# for awsIX in ((0,)):
    """ Basic AWS info """
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))

    closest = C[awsIX][0]
#     print awsLatLon, closest
#     print closest['lat'][0],closest['lon'][0]
    dx = "%.1f km" % closest[-1]
    print awsName, dx
    Tnew[:,:,awsIX] = tas[:,:,closest['ixLat'],closest['ixLon']]

12 4 47 99
Swiss Camp 28.1 km
Crawford Point1 8.3 km
NASA-U 24.0 km
GITS 33.6 km
Humboldt 31.4 km
Summit 8.0 km
Tunu-N 26.0 km
DYE-2 19.2 km
JAR1 37.4 km
Saddle 36.2 km
South Dome 39.2 km
NASA-E 30.1 km
Crawford Point2 13.9 km
NGRIP 37.9 km
NASA-SE 23.0 km
KAR 14.0 km
JAR2 33.5 km
KULU 19.1 km
JAR3 32.8 km
Aurora 15.3 km
Petermann GL 37.2 km
Petermann ELA 30.2 km
NEEM 7.5 km


Create a netCDF file for the new data
-----

In [9]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Closest-point temperature data"
ncfile.period = yrs

mon_dim = ncfile.createDimension('month', None)
hr_dim = ncfile.createDimension('hour', nHour)
stn_dim = ncfile.createDimension('station', nSites)

monVar = ncfile.createVariable('month', np.float32, ('month',))
monVar.description = "Month of the year"
monVar[:] = month

hrVar = ncfile.createVariable('hour', np.float32, ('hour',))
hrVar.description = "Hour of the day"
hrVar[:] = hour

tasVar = ncfile.createVariable('tas', np.float32, ('month','hour','station'))
tasVar.units = "deg C"
tasVar.description = "Surface temperature, closest point"
tasVar.long_name = "Reference height temperature"
# tasVar[:,:] = Tnew
tasVar[:] = Tnew

ncfile.close()